In [1]:
import torch
import os
import models
from charge_trainer import ChargeTrainer
from ocpmodels.common import logger
from ocpmodels.common.registry import registry
from ocpmodels.common.utils import setup_logging
# from DeepDFT import densitymodel
from chg_utils import ProbeGraphAdder
setup_logging()

import matplotlib.pyplot as plt
from torch_geometric.data import Batch

In [2]:
# !pip install wandb --upgrade
import wandb
wandb.login()
import pprint

2022-07-18 14:09:00 (ERROR): Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: emsunshine. Use `wandb login --relogin` to force relogin


In [4]:
sweep_config = {
    'method':'bayes',
    'metric':{'name':'val/charge_mae', 'goal':'minimize'},
    'parameters':{
        
        'num_interactions':{
            'distribution':'int_uniform',
            'max': 6,
            'min': 1,
        },
        
        'atom_channels':{
            'distribution':'q_log_uniform_values',
            'min': 16,
            'max': 128,
            'q': 8,
        },
        
        'probe_channels':{
            'distribution':'q_log_uniform_values',
            'min': 16,
            'max': 128,
            'q': 8,
        },
        
        'batch_size':{
            'distribution':'q_log_uniform_values',
            'min': 1,
            'max': 16,
            'q': 2,
        },
        
        'atom_filters':{
            'distribution':'q_log_uniform_values',
            'min': 8,
            'max': 128,
            'q': 8,
        },
        
        'probe_filters':{
            'distribution':'q_log_uniform_values',
            'min': 8,
            'max': 128,
            'q': 8,
        },
        
        'atom_gaussians':{
            'distribution':'q_log_uniform_values',
            'min': 8,
            'max': 32,
            'q': 8,
        },
        
        'probe_gaussians':{
            'distribution':'q_log_uniform_values',
            'min': 8,
            'max': 128,
            'q': 8,
        },
        
        'cutoff': {
            'distribution':'int_uniform',
            'max': 4,
            'min': 2,
        },
    }
}

pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'val/charge_mae'},
 'parameters': {'atom_channels': {'distribution': 'q_log_uniform_values',
                                  'max': 128,
                                  'min': 16,
                                  'q': 8},
                'atom_filters': {'distribution': 'q_log_uniform_values',
                                 'max': 128,
                                 'min': 8,
                                 'q': 8},
                'atom_gaussians': {'distribution': 'q_log_uniform_values',
                                   'max': 32,
                                   'min': 8,
                                   'q': 8},
                'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 16,
                               'min': 1,
                               'q': 2},
                'cutoff': {'distribution': 'int_uniform', 'max': 6, 'min': 3},
                'num_intera

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="charge-density-models-sweeps")
print(sweep_id)

In [3]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        task = {
            'dataset': 'lmdb',
            'description': 'Initial test of training on charges',
            'type': 'regression',
            'metric': ['charge_mse', 'charge_mae'],
            'primary_metric': 'charge_mae',
            'labels': ['charge_vals'],
        }
        
        model = {
            'name': 'charge_model',
            'num_interactions': config.num_interactions,
            'atom_channels': config.atom_channels,
            'probe_channels': config.probe_channels,

            'atom_model_config': {
                'name': 'schnet_charge',
                'num_filters':config.atom_filters,
                'num_gaussians':config.atom_gaussians,
                'cutoff':6,
            },

            'probe_model_config': {
                'name': 'schnet_charge',
                'num_filters':config.probe_filters,
                'num_gaussians':config.probe_gaussians,
                'cutoff':config.cutoff,
            },
        }
        
        optimizer = {
            'optimizer': 'Adam',
            'batch_size': config.batch_size,
            'eval_batch_size': 10,
            'num_workers': 1,
            'lr_initial': 5e-4,
            'scheduler': "ReduceLROnPlateau",
            'mode': "min",
            'factor': 0.96,
            'patience': 1,
            'max_epochs': 300,
        }
        
        dataset = [
            {'src': '../chg/100/train', 'normalize_labels': False}, # train set 
            {'src': '../chg/100/val'}, # val set (optional)
            # {'src': train_src} # test set (optional - writes predictions to disk)
            ]
        
        trainer_config = {
            'trainer': 'charge',
            'identifier': 'sweep_run',
            'is_debug': False,
            'run_dir': './runs/',
            'print_every': 1,
            'seed': 2,
            'logger': 'wandb',
            'local_rank': 0,
            'amp': True,

            'cutoff': config.cutoff,
        }
        
        trainer = registry.get_trainer_class(
            trainer_config['trainer'])(task = task,
                                       model = model,
                                       dataset = dataset,
                                       optimizer = optimizer,
                                       **trainer_config)
        
        trainer.train()

In [ ]:
wandb.agent('charge-density-models-sweeps/c4ve9o2z', train, count=100)

2022-07-18 14:09:00 (INFO): Starting sweep agent: entity=None, project=None, count=100


wandb: Agent Starting Run: ublsk3ol with config:
wandb: 	atom_channels: 48
wandb: 	atom_filters: 24
wandb: 	atom_gaussians: 24
wandb: 	batch_size: 8
wandb: 	cutoff: 5
wandb: 	num_interactions: 2
wandb: 	probe_channels: 56
wandb: 	probe_filters: 96
wandb: 	probe_gaussians: 40
wandb: 	train_probes: 300
2022-07-18 14:09:01 (ERROR): Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: WARNING Ignored wandb.init() arg id when running a sweep.


amp: true
cmd:
  checkpoint_dir: ./runs/checkpoints/2022-07-18-14-09-04-sweep_run
  commit: f251a59
  identifier: sweep_run
  logs_dir: ./runs/logs/wandb/2022-07-18-14-09-04-sweep_run
  print_every: 1
  results_dir: ./runs/results/2022-07-18-14-09-04-sweep_run
  seed: 2
  timestamp_id: 2022-07-18-14-09-04-sweep_run
dataset:
  normalize_labels: false
  src: ../chg/100/train
gpus: 1
logger: wandb
model: charge_model
model_attributes:
  atom_channels: 48
  atom_model_config:
    cutoff: 5
    name: schnet_charge
    num_filters: 24
    num_gaussians: 24
  num_interactions: 2
  probe_channels: 56
  probe_model_config:
    cutoff: 5
    name: schnet_charge
    num_filters: 96
    num_gaussians: 40
noddp: false
optim:
  batch_size: 8
  eval_batch_size: 10
  factor: 0.96
  lr_initial: 0.0005
  max_epochs: 300
  mode: min
  num_workers: 1
  optimizer: Adam
  patience: 1
  scheduler: ReduceLROnPlateau
slurm: {}
task:
  dataset: lmdb
  description: Initial test of training on charges
  labels:
 

2022-07-18 14:09:15 (INFO): Loading dataset: lmdb
2022-07-18 14:09:15 (INFO): Loading model: charge_model
2022-07-18 14:09:15 (INFO): Loaded ChargeModel with 87875 parameters.
charge_mse: 2.17e-01, charge_mae: 4.24e-01, loss: 8.48e-01, lr: 5.00e-04, epoch: 8.33e-02, step: 1.00e+00
charge_mse: 2.44e-01, charge_mae: 4.24e-01, loss: 4.24e-01, lr: 5.00e-04, epoch: 1.67e-01, step: 2.00e+00
charge_mse: 1.56e-01, charge_mae: 3.39e-01, loss: 3.39e-01, lr: 5.00e-04, epoch: 2.50e-01, step: 3.00e+00
charge_mse: 1.94e-01, charge_mae: 2.59e-01, loss: 2.59e-01, lr: 5.00e-04, epoch: 3.33e-01, step: 4.00e+00
charge_mse: 1.92e-01, charge_mae: 1.98e-01, loss: 1.98e-01, lr: 5.00e-04, epoch: 4.17e-01, step: 5.00e+00
charge_mse: 1.69e-01, charge_mae: 1.68e-01, loss: 1.68e-01, lr: 5.00e-04, epoch: 5.00e-01, step: 6.00e+00
charge_mse: 1.19e-01, charge_mae: 1.10e-01, loss: 1.10e-01, lr: 5.00e-04, epoch: 5.83e-01, step: 7.00e+00
charge_mse: 1.23e-01, charge_mae: 1.53e-01, loss: 1.53e-01, lr: 5.00e-04, epoch: 6

...

2022-07-18 23:46:07 (INFO): charge_mse: 0.0854, charge_mae: 0.0406, loss: 0.0406, epoch: 175.0000


charge_mse: 6.45e-03, charge_mae: 1.80e-02, loss: 1.80e-02, lr: 3.67e-05, epoch: 1.75e+02, step: 4.03e+03
charge_mse: 2.17e-02, charge_mae: 2.73e-02, loss: 2.73e-02, lr: 3.67e-05, epoch: 1.75e+02, step: 4.03e+03
charge_mse: 1.13e-01, charge_mae: 3.98e-02, loss: 3.98e-02, lr: 3.67e-05, epoch: 1.75e+02, step: 4.03e+03
charge_mse: 6.92e-03, charge_mae: 1.77e-02, loss: 1.77e-02, lr: 3.67e-05, epoch: 1.75e+02, step: 4.03e+03
charge_mse: 8.54e-02, charge_mae: 3.00e-02, loss: 3.00e-02, lr: 3.67e-05, epoch: 1.75e+02, step: 4.03e+03
charge_mse: 4.81e-02, charge_mae: 3.39e-02, loss: 3.39e-02, lr: 3.67e-05, epoch: 1.75e+02, step: 4.03e+03
charge_mse: 9.21e-03, charge_mae: 1.98e-02, loss: 1.98e-02, lr: 3.67e-05, epoch: 1.75e+02, step: 4.03e+03
charge_mse: 4.91e-03, charge_mae: 1.36e-02, loss: 1.36e-02, lr: 3.67e-05, epoch: 1.75e+02, step: 4.03e+03
charge_mse: 1.72e-02, charge_mae: 2.75e-02, loss: 2.75e-02, lr: 3.67e-05, epoch: 1.75e+02, step: 4.03e+03
charge_mse: 2.87e-03, charge_mae: 1.92e-02, lo

device 0: 100%|██████████| 1/1 [00:08<00:00,  8.02s/it]

2022-07-18 23:47:11 (INFO): charge_mse: 0.1008, charge_mae: 0.0412, loss: 0.0412, epoch: 176.0000


charge_mse: 4.17e-02, charge_mae: 2.96e-02, loss: 2.96e-02, lr: 3.52e-05, epoch: 1.76e+02, step: 4.05e+03
charge_mse: 5.31e-03, charge_mae: 1.91e-02, loss: 1.91e-02, lr: 3.52e-05, epoch: 1.76e+02, step: 4.05e+03
charge_mse: 7.07e-02, charge_mae: 4.85e-02, loss: 4.85e-02, lr: 3.52e-05, epoch: 1.76e+02, step: 4.05e+03
charge_mse: 6.11e-03, charge_mae: 2.24e-02, loss: 2.24e-02, lr: 3.52e-05, epoch: 1.76e+02, step: 4.05e+03
charge_mse: 2.78e-02, charge_mae: 2.78e-02, loss: 2.78e-02, lr: 3.52e-05, epoch: 1.76e+02, step: 4.05e+03
charge_mse: 4.44e-02, charge_mae: 1.98e-02, loss: 1.98e-02, lr: 3.52e-05, epoch: 1.76e+02, step: 4.05e+03
charge_mse: 4.64e-03, charge_mae: 1.65e-02, loss: 1.65e-02, lr: 3.52e-05, epoch: 1.76e+02, step: 4.06e+03
charge_mse: 1.26e-02, charge_mae: 2.07e-02, loss: 2.07e-02, lr: 3.52e-05, epoch: 1.76e+02, step: 4.06e+03
charge_mse: 8.68e-02, charge_mae: 4.20e-02, loss: 4.20e-02, lr: 3.52e-05, epoch: 1.76e+02, step: 4.06e+03
charge_mse: 2.50e-02, charge_mae: 2.71e-02, lo